
# Анализ данных Global Superstore

Этот проект анализирует данные из Global Superstore для ответа на следующие вопросы:

### Общие вопросы:
1. **Какова общая сумма продаж за весь период?**
   - Это помогает понять общий доход, полученный магазином.

2. **Сколько заказов было выполнено за каждый год?**
   - Это помогает анализировать рост или спад количества заказов со временем.

### Анализ по категориям:
3. **Какие категории продуктов приносят наибольший доход?**
   - Определение наиболее прибыльных категорий помогает в стратегиях по управлению запасами и маркетингу.

4. **Какие регионы являются самыми прибыльными?**
   - Это помогает нацеливать маркетинговые усилия и распределение ресурсов в наиболее прибыльные регионы.

### Временной анализ:
5. **Как изменялись продажи по месяцам и годам?**
   - Понимание тенденций продаж со временем может информировать о сезонных маркетинговых стратегиях.

6. **Какова сезонность продаж?**
   - Определение пиковых и низких периодов помогает в планировании акций и запасов.

### Анализ клиентов:
7. **Какие клиенты делают наибольшее количество заказов?**
   - Определение лояльных клиентов может помочь в персонализированном маркетинге и стратегиях удержания клиентов.

8. **Какова средняя стоимость заказа для каждого клиента?**
   - Понимание поведения клиентов по расходам может направлять стратегии ценообразования и скидок.


In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Создаем соединение с базой данных SQLite
engine = create_engine('sqlite:///global_superstore.db')
connection = engine.connect()

In [3]:
# Загрузка данных из файла
data = pd.read_csv('data/Global Superstore.txt', delimiter='\t')

In [25]:
# Разделение данных для вставки в таблицы
customers_data = data[['Customer ID', 'Customer Name', 'Segment']].drop_duplicates()
orders_data = data[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID']].drop_duplicates()
products_data = data[['Product ID', 'Category', 'Sub-Category']].drop_duplicates()
sales_data = data[['Order ID', 'Product ID', 'Sales', 'Discount', 'Shipping Cost', 'Market', 'Country', 'State', 'City', 'Year']]

In [5]:
# Вставка данных в таблицы
customers_data.to_sql('Customers', engine, if_exists='append', index=False)
orders_data.to_sql('Orders', engine, if_exists='append', index=False)
products_data.to_sql('Products', engine, if_exists='append', index=False)
sales_data.to_sql('Sales', engine, if_exists='append', index=False)

51290

In [15]:
# Вопрос 1: Какова общая сумма продаж за весь период?
query1 = "SELECT SUM(SalesAmount) AS TotalSales FROM Sales;"
result1 = connection.execute(query1).fetchall()
total_sales = pd.DataFrame(result1, columns=["TotalSales"])
total_sales

,TotalSales
0,12642905


In [14]:
# Вопрос 2: Сколько заказов было выполнено за каждый год?
query2 = "SELECT strftime('%Y', OrderDate) AS Year, COUNT(OrderID) AS TotalOrders FROM Orders GROUP BY Year;"
result2 = connection.execute(query2).fetchall()
total_orders_per_year = pd.DataFrame(result2, columns=["Year", "TotalOrders"])
total_orders_per_year

,Year,TotalOrders
0,2011,4516
1,2012,5480
2,2013,6890
3,2014,8868


In [13]:
# Вопрос 3: Какие категории продуктов приносят наибольший доход?
query3 = "SELECT Products.Category, SUM(Sales.SalesAmount) AS TotalSales FROM Sales JOIN Products ON Sales.ProductID = Products.ProductID GROUP BY Products.Category ORDER BY TotalSales DESC;"
result3 = connection.execute(query3).fetchall()
top_product_categories = pd.DataFrame(result3, columns=["Category", "TotalSales"])
top_product_categories

,Category,TotalSales
0,Technology,4744691
1,Furniture,4110884
2,Office Supplies,3787698


In [26]:
# Вопрос 4: Какие регионы являются самыми прибыльными (Топ-10)?
query4 = "SELECT Country, SUM(SalesAmount) AS TotalSales FROM Sales GROUP BY Country ORDER BY TotalSales DESC LIMIT 10;"
result4 = connection.execute(query4).fetchall()
top_profitable_countries = pd.DataFrame(result4, columns=["Country", "TotalSales"])
top_profitable_countries

,Country,TotalSales
0,United States,2297354
1,Australia,925257
2,France,858930
3,China,700591
4,Germany,628857
5,Mexico,622620
6,India,589664
7,United Kingdom,528570
8,Indonesia,404887
9,Brazil,361098


In [18]:
# Вопрос 5: Как изменялись продажи по месяцам и годам?
query5 = """
SELECT strftime('%Y', OrderDate) AS Year, 
       strftime('%m', OrderDate) AS Month, 
       SUM(SalesAmount) AS TotalSales 
FROM Sales 
JOIN Orders ON Sales.OrderID = Orders.OrderID 
GROUP BY Year, Month 
ORDER BY Year, Month;
"""
query5 = "SELECT strftime('%Y', OrderDate) AS Year, strftime('%m', OrderDate) AS Month, SUM(SalesAmount) AS TotalSales FROM Sales JOIN Orders ON Sales.OrderID = Orders.OrderID GROUP BY Year, Month ORDER BY Year, Month;"
result5 = connection.execute(query5).fetchall()
sales_trends = pd.DataFrame(result5, columns=["Year", "Month", "TotalSales"])
sales_trends

,Year,Month,TotalSales
0,2011,01,102561
1,2011,02,97257
2,2011,03,147910
3,2011,04,121100
4,2011,05,151716
5,2011,06,225679
6,2011,07,123971
7,2011,08,229149
8,2011,09,296512
9,2011,10,202470


In [24]:
# Вопрос 6: Какова сезонность продаж?
query6 = "SELECT strftime('%m', OrderDate) AS Month, SUM(SalesAmount) AS TotalSales FROM Sales JOIN Orders ON Sales.OrderID = Orders.OrderID GROUP BY Month ORDER BY Month;"
result6 = connection.execute(query6).fetchall()
seasonality_sales = pd.DataFrame(result6, columns=["Month", "TotalSales"])
seasonality_sales

,Month,TotalSales
0,01,720974
1,02,583641
2,03,805463
3,04,746183
4,05,945807
5,06,1369909
6,07,805129
7,08,1396139
8,09,1527837
9,10,1199989


In [23]:
# Вопрос 7: Какие клиенты делают наибольшее количество заказов (Топ-10)?
query7 = "SELECT Customers.CustomerName, COUNT(Orders.OrderID) AS TotalOrders FROM Orders JOIN Customers ON Orders.CustomerID = Customers.CustomerID GROUP BY Customers.CustomerName ORDER BY TotalOrders DESC LIMIT 10;"
result7 = connection.execute(query7).fetchall()
top_customers = pd.DataFrame(result7, columns=["CustomerName", "TotalOrders"])
top_customers

,CustomerName,TotalOrders
0,Michael Paige,47
1,Frank Olsen,47
2,Anna Andreadi,47
3,Sara Luxemburg,46
4,Laura Armstrong,46
5,Kristen Hastings,46
6,Tracy Blumstein,45
7,Patrick O'Brill,45
8,Noel Staavos,45
9,Joseph Holt,45


In [22]:
# Вопрос 8: Какова средняя стоимость заказа для каждого клиента?
query8 = "SELECT Customers.CustomerName, AVG(Sales.SalesAmount) AS AvgOrderValue FROM Sales JOIN Orders ON Sales.OrderID = Orders.OrderID JOIN Customers ON Orders.CustomerID = Customers.CustomerID GROUP BY Customers.CustomerName ORDER BY AvgOrderValue DESC;"
result8 = connection.execute(query8).fetchall()
avg_order_value_per_customer = pd.DataFrame(result8, columns=["CustomerName", "AvgOrderValue"])
avg_order_value_per_customer

,CustomerName,AvgOrderValue
0,Sean Miller,677.019231
1,Hunter Lopez,560.555556
2,Christopher Conant,473.480000
3,Tom Ashbrook,469.311111
4,Mike Gockenbach,456.769231
...,...,...
790,Sung Shariari,121.590909
791,Karen Seio,119.595745
792,Catherine Glotzbach,117.600000
793,Allen Armold,117.089286
